In [1]:
from glass.cls.eval import mtx_binary_class

mtx_binary_class(
    '/mnt/g/otol_exp2/results/osm_green_vali_exp1.shp',
    'clsarea',
    'classi',
    '14100', '31000',
    '/mnt/g/otol_exp2/results/exp1_vali_refarea.xlsx'
)

'/mnt/g/otol_exp2/results/exp1_vali_refarea.xlsx'

In [ ]:
shp = '/mnt/g/otol_exp2/guavsforest.gdb/calibra/ua18_forest_evu'

fid = 'ffif'

refcol = 'code_2018'

evu_cls = '14100'
for_cls = '31000'

cols = {
    'urban_area'  : 'aurbbup',
    'directions'  : 'dirbup',
    'urban_green' : 'edievu_bup'
}

out = '/mnt/d/osm2lulc01/results/ua_class_exp4.shp'

In [ ]:
import numpy as np
from glass.rd.shp import shp_to_obj
from glass.wt.shp import df_to_shp

In [ ]:
gdf = shp_to_obj(shp)

In [ ]:
gdf["classscore"] = 0

In [ ]:
# Positive value = EVU
# Negative value = Forest

# Area > 1 = Forest
gdf["classscore"] = np.where(
    gdf['garea'] > 1, gdf.classscore - 1,
    gdf.classscore + 1
)

# Area > 10 = More Forest
gdf["classscore"] = np.where(
    gdf['garea'] >= 30, gdf.classscore - 1,
    gdf.classscore
)

# Area > 30 = Even More Forest
gdf["classscore"] = np.where(
    gdf['garea'] >= 100, gdf.classscore - 1,
    gdf.classscore
)

# Directions >= 5 = GUA
gdf["classscore"] = np.where(
    gdf[cols['directions']] >= 5, gdf.classscore + 1,
    gdf.classscore - 1
)

gdf["classscore"] = np.where(
    gdf[cols['directions']] == 8, gdf.classscore + 1,
    gdf.classscore
)

# Built up area >= 6 = GUA
gdf["classscore"] = np.where(
    gdf[cols['urban_area']] >= 6, gdf.classscore + 1,
    gdf.classscore - 1
)

# Number of EVU points >= 1 GUA
gdf["classscore"] = np.where(
    gdf['npnts_evu'] >= 1, gdf.classscore + 2,
    gdf.classscore
)

# Length of EVU Lines >= 1 GUA
gdf["classscore"] = np.where(
    gdf['lenlines_evu'] >= 1, gdf.classscore + 2,
    gdf.classscore
)

# % Area of EVU Polygons >= 30 GUA
gdf["classscore"] = np.where(
    gdf['poly_evu'] >= 30, gdf.classscore + 2,
    gdf.classscore
)

# Number of Forest points >= 5 Forest
gdf["classscore"] = np.where(
    gdf['npnts_for'] >= 5, gdf.classscore - 2,
    gdf.classscore
)

# Length of Forest Lines >= 1 Forest
gdf["classscore"] = np.where(
    gdf['lenlines_for'] >= 1, gdf.classscore - 2,
    gdf.classscore
)

# Population Density >= 1250 GUA
gdf["classscore"] = np.where(
    gdf['denspop'] >= 1250, gdf.classscore + 1,
    gdf.classscore - 1
)

gdf["classscore"] = np.where(
    gdf['denspop'] >= 5000, gdf.classscore + 1,
    gdf.classscore
)

gdf["classscore"] = np.where(
    gdf['denspop'] >= 10000, gdf.classscore + 1,
    gdf.classscore
)

# Number of green polygons inside the built up >= 1 = GUA
gdf["classscore"] = np.where(
    gdf[cols['urban_green']] >= 1, gdf.classscore + 1,
    gdf.classscore - 1
)

# Tree cover density
# >=20 is forest
gdf["classscore"] = np.where(
    gdf['tcd_mean'] >= 20, gdf.classscore - 1,
    gdf.classscore + 1
)

In [ ]:
gdf["class"] = np.where(
    gdf.classscore < 0, '31000', np.where(
        gdf.classscore > 0, '14100', '00000'
    )
)

gdf['class'] = np.where(
    (gdf.classscore == 0) & (gdf['garea'] > 1),
    '31000', np.where(
        (gdf.classscore == 0) & (gdf['garea'] <= 1),
        '14100', gdf['class']
    )
)

gdf['class'] = np.where(
    gdf['garea'] >= 100, '31000', gdf['class']
)

In [ ]:
df_to_shp(gdf, out)